In [1]:
%reload_ext autoreload
%autoreload 2

from configData import configVars
from transcriptLoader import TranscriptData

# videoNames = [
#     "New Quizzes Video",
#     "Rearrange Playlist video",
#     "IMSE 514 Presentation",
#     "New Google Assignments in Canvas",
#     "Piazza Introduction Workshop",
# ]

config = configVars()
config.videoToUse = "New Quizzes Video"
config.setFromEnv()


videoData = TranscriptData(config)

INFO:root:All configuration parameters set up successfully.
INFO:root:Transcript data extracted from Captions\New Quizzes Video\new_quizzes_1_xzqdjesb.srt
INFO:root:Transcript data shape: (1947, 3)
INFO:root:Transcript data head:                                        Line                   Start  \
0                         Hello, my name is 1900-01-01 00:00:03.140   
1  Monica Hicks and welcome to new quizzes. 1900-01-01 00:00:05.910   
2  Joining the future of quizzes in Canvas. 1900-01-01 00:00:08.010   
3                  We will be recording and 1900-01-01 00:00:10.710   
4       I will be sharing their recordings. 1900-01-01 00:00:13.094   

                      End  
0 1900-01-01 00:00:05.910  
1 1900-01-01 00:00:08.010  
2 1900-01-01 00:00:10.710  
3 1900-01-01 00:00:13.094  
4 1900-01-01 00:00:15.239  
INFO:root:Combined Transcript data shape: (160, 3)
INFO:root:Combined Transcript data head:                                       Combined Lines                   Start  \
0  

In [ ]:
from transcriptLoader import retrieveTranscript

videoData = retrieveTranscript(config, overwrite=True)
videoData.printTranscript()

In [2]:
from topicExtractor import retrieveTopics

topicModeller = retrieveTopics(config, videoData, overwrite=True)
topicModeller.printTopics()

INFO:root:Generating & saving Topic Model and Data...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Generating & saving Topic Model and Data...


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:httpx:HTTP Request: POST https://api.umgpt.umich.edu/azure-openai-api/openai/deployments/gpt-4/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.umgpt.umich.edu/azure-openai-api/openai/deployments/gpt-4/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.umgpt.umich.edu/azure-openai-api/openai/deployments/gpt-4/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.umgpt.umich.edu/azure-openai-api/openai/deployments/gpt-4/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.umgpt.umich.edu/azure-openai-api/openai/deployments

Topic Extraction Token Count: 24926
Topic Model and Data generated and saved for current configuration.
{-1: 'Online Quiz Creation Instructions', 0: 'Canvas New Quizzes Update', 1: 'Quiz Points Configuration Guidance', 2: 'Student Accommodations Management', 3: 'Educational Assessment Design'}


In [3]:
from questionGenerator import retrieveQuestions

questionData = retrieveQuestions(config, topicModeller, videoData, overwrite=True)
questionData.printQuestions()

INFO:root:Clustered Topics data shape: (11, 7)
INFO:root:Clustered Topics head:    Topic                                              Words  Frequency  \
0      0                        Canvas New Quizzes Training          3   
1      1            Educational Quiz Platforms Instructions          1   
2      0                   Educational Quiz Platform Update          3   
3      3                       Educational Assessment Tools          2   
4      0  Educational Platform Features Tutorial, Educat...          3   

                      Timestamp                         Start  \
0 1899-12-31 23:59:58.134021000 1900-01-01 00:00:03.140000000   
1 1899-12-31 23:59:58.134021000 1899-12-31 23:59:58.134021000   
2 1900-01-01 00:02:05.237048780 1900-01-01 00:02:05.237048780   
3 1900-01-01 00:04:07.334097560 1900-01-01 00:04:07.334097560   
4 1900-01-01 00:04:07.334097560 1900-01-01 00:04:07.334097560   

                            End                         Topic Title  
0 1900-01-01 0

Question Generation Token Count: 9084
Question Data generated and saved for current configuration.
Topic: Canvas New Quizzes Update
Insert Point: 00:02:05
Question: Who owns Canvas?
Answers: Instructure
	Google
	Microsoft
	Apple
Correct: Instructure
Reason: Instructure is the correct answer because the transcript explicitly states that 'Canvas is owned by Instructure.' Google, Microsoft, and Apple are incorrect because they are not mentioned in the context of owning Canvas.
------------------------------------
Topic: Educational Assessment Design
Insert Point: 00:12:15
Question: Which of the following is NOT a feature of the new quizzes as described in the transcript?
Answers: Ability to analyze quiz data with item analysis tables
	Option to shuffle questions and answers for each quiz
	On-screen calculators for students during the quiz
	Assessable hotspot questions
Correct: Assessable hotspot questions
Reason: The transcript mentions that hotspot questions are not assessable, which is 